In [ ]:
import serial
import time
import numpy as np
from airfoil.cnc import CNC
from airfoil.util.path_planning import compensate_feedrate
from airfoil.power_supply import KORAD_KD3005P
import pandas as pd

In [ ]:
import winsound
def bep(f:float=0.0,ms:int=500):
    winsound.Beep(int(440 * 2.0**(f/12.0)),ms)

In [ ]:
cnc = CNC()

In [ ]:
cnc.home()
cnc.metric()
cnc.absolute()

In [ ]:
cnc.travel(100,50,100,50)

In [ ]:
cnc.set_position(0,0,0,0)
cnc.metric()
cnc.absolute()

In [ ]:
ps = KORAD_KD3005P("COM11")

In [ ]:
ps.status()

In [ ]:
ps.output_off()

In [ ]:
cnc.travel(0,0,0,0)

In [ ]:
TIME = 10
SAMPLES = 20
DISTANCE = 30
IMIN = 1.5
IMAX = 2.5
FEED = compensate_feedrate(1,0,1,0) * DISTANCE/(TIME/60)
print(f"""
    {TIME=:.2f}
    TARGET_FEED={DISTANCE/(TIME/60):.2f}
    {FEED=:.2f}
    {TIME/SAMPLES=:.2f}
""")

In [ ]:
cuts = 10
spacing = 4
Imin = 1.5
Imax = 2.5
target_feed_rate = 200
compensated_feedrate = compensate_feedrate(1,0,1,0) * target_feed_rate
start_positions = np.arange(cuts)*spacing
currents = np.linspace(Imin,Imax, cuts)
depth = 15
print(f"""
    height = {start_positions[-1]:.0f}
    {target_feed_rate = :.0f}
    {compensated_feedrate = :.1f}
    [{", ".join(f"{f:.2f}" for f in currents)}]
""")


In [ ]:
bep(13)
ps.output_on()
for current, start_position in reversed(list(zip(currents,start_positions))):
    cnc.absolute()
    cnc.serial.read_until("ok\r\n")
    ps.set_current(current)
    time.sleep(0.050)
    bep(12)
    cnc.travel(0,start_position,0,start_position)
    cnc.serial.read_until("ok\r\n")
    cnc.relative()
    cnc.serial.read_until("ok\r\n")
    cnc.feed(compensated_feedrate,10,0,10,0)
    cnc.serial.read_until("ok\r\n")
    bep(11)
    ps.set_current(1.0)
    cnc.feed(compensated_feedrate,-10,0,-10,0)
    cnc.serial.read_until("ok\r\n")
ps.output_off()
bep(10)

In [ ]:
cnc.alarm_soft_reset()

In [ ]:
ps.output_off()

In [ ]:
cnc.travel(0,0,0,0)

In [ ]:
np.linspace(200,300)
cnc.relative()
cnc.feed(FEED,10,)

In [ ]:
capture = []

In [ ]:
ps.set_current(1.0)
time.sleep(0.1)
bep(13)
ps.output_on()
time.sleep(0.2)
capture.append(ps.status())
time.sleep(0.1)
ps.output_off()
bep(10)
pd.DataFrame.from_records(capture)

In [ ]:
time.sleep(3)
bep(10)
ps.set_voltage(15)
time.sleep(0.050)

ps.set_current(IMIN)
time.sleep(0.050)

time.sleep(3)
bep(13)
ps.output_on()
time.sleep(0.050)
cnc.feed(FEED,DISTANCE,0,DISTANCE,0)
for I in np.linspace(IMIN,IMAX,SAMPLES):
    ps.set_current(I)
    time.sleep(TIME/SAMPLES)
ps.output_off()
bep(10)

## Falldown Test

In [ ]:
total_time = 8
currents = np.linspace(2.5,1.5, 20)
interval = total_time/len(currents)
interval

In [ ]:
bep(13)
ps.set_voltage(15)
time.sleep(0.050)
ps.set_current(0.1)
time.sleep(0.050)
ps.output_on()
time.sleep(0.050)
for current,note in zip(currents,np.arange(len(currents))):
    ps.set_current(current)
    bep(14+note, int(interval*1000))
ps.output_off()
bep(10)

In [ ]:
ps.output_off()

In [ ]:
12/30

In [ ]:
[f"{i:.2f}" for i in currents]